In [1]:
import sys
import os
import numpy
import pandas
import json   

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import decomposition


# print 'Argument List:', str(sys.argv)
#
# if(len(args)!=6){
#   stop("Wrong number of arguments. Usage:\nTrain_Evaluate_7MLP-100PCA.py <label> <target-variable> <data-sources> <train-set-sessions> <test-set-sessions>")
# }
#
# label = sys.argv[1]
# target = sys.argv[2] # either 'Activity' or 'Social'
# datasourcestring = sys.argv[3] # Comma separated combinations of all,et,acc,aud,vid
# trainstring = sys.argv[4]
# teststring = sys.argv[5]
# For tests
label='7MLP-100PCA_et_GM_LOSO_Activity_1'
target='Activity'
datasourcestring='et,acc'
trainstring='case1-day1-session2-teacher1,case1-day1-session3-teacher1,case1-day1-session4-teacher1,case2-day1-session1-teacher2,case2-day1-session2-teacher2,case2-day2-session1-teacher2,case2-day2-session2-teacher2,case2-day3-session1-teacher2,case2-day3-session2-teacher2,case2-day4-session1-teacher2,case2-day4-session2-teacher2'
teststring='case1-day1-session1-teacher1'


Using Theano backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# We parse the data sources to take into account, and sessions for the train and test sets
features = range(0,2) # These are only the session and timestamp
sources = datasourcestring.split(",")
for source in sources:
    if(source=='all'):
        features.extend(range(2,7557))
        break
    elif(source=='et'):
        features.extend(range(2,12))
    elif(source=='acc'):
        features.extend(range(12,152))
    elif(source=='aud'):
        features.extend(range(152,6557))
    elif(source=='vid'):
        features.extend(range(6557,7557))
    else:
        sys.exit("Wrong data sources. Possible values: all,et,acc,aud,vid")
features.extend(range(7557,7559)) # Add activity and Social
print("Selected features: "+str(len(features)))

sessiontrain = trainstring.split(",") # Gives an array of the sessions to train in
sessiontest = teststring.split(",") # Gives an array of the sessions to train in

if(len(sessiontrain)==0 | len(sessiontest)==0):
    sys.exit("Wrong train/test sessions specification. Should be a comma-separated string with the sessions identificators")



Selected features: 154


In [3]:
# TODO: To be tested with the python script
path = os.path.dirname(os.path.realpath(sys.argv[0]))

In [4]:
# READING AND PREPARING THE DATA
#processeddatadir = path
processeddatadir = "../src/models" # TODO: Change this for the actual script
datafile = os.path.join(processeddatadir,'completeDataset.csv')
gzdatafile = os.path.join(processeddatadir,'completeDataset.csv.gz')
fulldata = pandas.DataFrame()
if(os.path.isfile(datafile)):
    fulldata = pandas.read_csv(datafile, sep=',', quotechar='"')
elif(os.path.isfile(gzdatafile)):
    fulldata = pandas.read_csv(gzdatafile, compression='gzip', sep=',', quotechar='"')
else:
    sys.exit("Data not available in the script's folder")

# Drop the useless first column
fulldata.drop(fulldata.columns[[0]],axis=1,inplace=True)


In [5]:
def cleanAct(value):
    if pandas.isnull(value):
        return 'Other'
    elif value=='OFF' or value=='TDT' or value=='TEC':
        return 'Other'
    else:
        return value

def cleanSoc(value):
    if pandas.isnull(value):
        return 'Other'
    else:
        return value


# We only look for predicting 4 states of activity and 3 of social, the rest (incl.NA) we bunch in 'Other' (so in the end it is a 5- and 4-class classification problem)
fulldata['Activity'] = fulldata['Activity.win'].map(cleanAct)
fulldata['Social'] = fulldata['Social.win'].map(cleanSoc)

# Drop the useless first column
fulldata.drop(fulldata.columns[[2,3,4]],axis=1,inplace=True)
print(fulldata.columns.values[0:5],"...",fulldata.columns.values[-5:])
fulldata.head(3)

(array(['session', 'timestamp', 'value.Mean', 'value.SD', 'value.Fix'], dtype=object), '...', array(['V998', 'V999', 'V1000', 'Activity', 'Social'], dtype=object))


,session,timestamp,value.Mean,value.SD,value.Fix,value.Sac,value.Fix.Dur,value.Fix.Disp,value.Sac.Dur,value.Sac.Amp,value.Sac.Len,value.Sac.Vel,value.X.Mean,value.X.SD,value.X.Max,value.X.Min,value.X.Median,value.X.FFT.1,value.X.FFT.2,value.X.FFT.3,
0,case1-day1-session1-teacher1,5000,3.731296,0.610973,0,0.271604,155.517241,103.428029,77.047619,22.980952,210.109938,2.842520,1.968844,3.606027,13.810,-5.660,0.7560,3.937689,1.596099,2.501288,...
1,case1-day1-session1-teacher1,10000,3.623267,0.579418,0,0.294837,195.000000,140.122022,78.210526,25.407895,221.507133,2.896652,0.820018,4.076379,13.810,-9.864,0.0760,1.640036,3.756661,2.473409,...
2,case1-day1-session1-teacher1,15000,3.663333,0.578927,0,0.230081,193.250000,114.539547,75.232558,19.411628,171.338448,2.230033,-1.459000,2.536654,10.151,-9.864,-1.5845,2.918000,0.494229,1.064934,...


In [6]:
# Now the column indices match what is expected in the arguments parsed above
# * [,0]: ''session id''
# * [,1]: ''timestamp'' within the session (in ms)
# * [,2:12]: ''eyetracking'' features (mean/sd pupil diameter, nr. of long fixations, avg. saccade speed, fixation duration, fixation dispersion, saccade duration, saccade amplitude, saccade length, saccade velocity)
# * [,12:152]: ''accelerometer'' features, including X, Y, Z (mean, sd, max, min, median, and 30 FFT coefficients of each of them) and jerk (mean, sd, max, min, median, and 30 FFT coefficients of each of it)
# * [,152:6557]: ''audio'' features extracted from an audio snippet of the 10s window, using openSMILE. Includes features about whether there is someone speaking (153:163), emotion recognition models (164:184), and brute-force audio spectrum features and characteristics used in various audio recognition challenges/tasks (185:6557)
# * [,6557:7557]: ''video'' features extracted from an image taken in the middle of the window (the 1000 values of the last layer when passing the immage through a VGG pre-trained model)
# * [,7557:7559]: ''Activity,Social'' labels we want to predict


# SELECTING THE DATASET FEATURES (DATA SOURCES BEING TRIED)
data = fulldata.ix[:,features]

# We drop the non-needed target variable
if target == 'Activity':
    data.drop('Social',axis=1,inplace=True)
elif target == 'Social':
    data.drop('Activity',axis=1,inplace=True)   

print(data.shape)
data.head(3)

(5561, 153)


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame


,session,timestamp,value.Mean,value.SD,value.Fix,value.Sac,value.Fix.Dur,value.Fix.Disp,value.Sac.Dur,value.Sac.Amp,value.Sac.Len,value.Sac.Vel,value.X.Mean,value.X.SD,value.X.Max,value.X.Min,value.X.Median,value.X.FFT.1,value.X.FFT.2,value.X.FFT.3,
0,case1-day1-session1-teacher1,5000,3.731296,0.610973,0,0.271604,155.517241,103.428029,77.047619,22.980952,210.109938,2.842520,1.968844,3.606027,13.810,-5.660,0.7560,3.937689,1.596099,2.501288,...
1,case1-day1-session1-teacher1,10000,3.623267,0.579418,0,0.294837,195.000000,140.122022,78.210526,25.407895,221.507133,2.896652,0.820018,4.076379,13.810,-9.864,0.0760,1.640036,3.756661,2.473409,...
2,case1-day1-session1-teacher1,15000,3.663333,0.578927,0,0.230081,193.250000,114.539547,75.232558,19.411628,171.338448,2.230033,-1.459000,2.536654,10.151,-9.864,-1.5845,2.918000,0.494229,1.064934,...


In [7]:
# SPLITTING THE DATA
test = data.loc[data['session'].isin(sessiontest)]
train = data.loc[data['session'].isin(sessiontrain)]
print(test.shape)
print(train.shape)
# Removing null values
test = test[test.notnull().all(axis=1)]
train = train[train.notnull().all(axis=1)]
print("Removing null and NAs...")
print(test.shape)
print(train.shape)

(461, 153)
(5100, 153)
Removing null and NAs...
(455, 153)
(5065, 153)


In [8]:
X_train = train.values[:,range(2,train.shape[1]-1)].astype(float)
Y_train = train.values[:,(train.shape[1]-1)]
X_test = test.values[:,range(2,test.shape[1]-1)].astype(float)
Y_test = test.values[:,(test.shape[1]-1)]
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

((5065, 150), (5065,), (455, 150), (455,))


In [9]:
#######################################################
# DO OTHER DATA TRANSFORMATIONS NEEDED, e.g. PCA, SELECT K-BEST FEATURES, etc (NORMALLY, ON THE TRAIN SET ONLY, TO BE APPLIED LATER TO THE TEST SET)
print("Transforming data... ")
k = 100

# We standardize on the basis of the training data
scaler = StandardScaler().fit(X_train)
X_train_st = scaler.transform(X_train)
X_test_st = scaler.transform(X_test)

if X_train.shape[1]>k:
    # Apply 100-component pca
    print("PCA with "+str(k)+" components")
    pca = decomposition.PCA(n_components=k)
    X_train_pca = pca.fit_transform(X_train_st)
    X_test_pca = pca.transform(X_test_st)
    print 'Variance explained:'
    print pca.explained_variance_ratio_
    print 'Total variance explained by 100 components:'
    print sum(pca.explained_variance_ratio_)
else:
    X_train_pca = X_train_st
    X_test_pca = X_test_st
    
#######################################################

Transforming data... 
PCA with 100 components
Variance explained:
[ 0.44190644  0.05668825  0.03675999  0.02993198  0.02593058  0.02333469
  0.02025391  0.01558931  0.0133012   0.0117185   0.01119025  0.01064468
  0.00983347  0.008785    0.00820497  0.00727277  0.00671948  0.00647124
  0.00621214  0.00595496  0.00586301  0.00527729  0.00501091  0.00468493
  0.00460191  0.00454919  0.00435207  0.00417185  0.00404601  0.00390369
  0.00384401  0.00372716  0.00369096  0.00359745  0.00350342  0.00346141
  0.00337186  0.0033182   0.00319907  0.00315617  0.00311213  0.00306549
  0.00303895  0.00301095  0.00295121  0.00288408  0.0028756   0.0028414
  0.00279251  0.00276759  0.00271181  0.00264207  0.00260798  0.00256459
  0.00254773  0.00247843  0.00244272  0.00242012  0.00240077  0.0023876
  0.00235121  0.00230202  0.00223458  0.00221437  0.0022023   0.0021865
  0.00215707  0.00212917  0.00209129  0.00206304  0.002038    0.00202478
  0.00200438  0.00195956  0.00190448  0.00189455  0.00188458 

In [10]:
# PREPARING THE DATA FOR KERAS TRAINING
# One hot encoding of the response variable (using dummy variables)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y_train = encoder.transform(Y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = to_categorical(encoded_Y_train)
encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = to_categorical(encoded_Y_test)


# Apply dropout regularization, it is overfitting!
def create_deeper_dropout_decay_PCA(k):
    # create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(k,)))
    model.add(Dense(300, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(300, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(80, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(80, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(20, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(20, init='uniform', activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(5, init='uniform', activation='sigmoid'))
    # Compile model, with larger learning rate and momentum, as recommended by the original paper
    sgd = SGD(lr=0.1, momentum=0.8, decay=0.0001, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

# evaluate baseline model with standardized dataset
# fix random seed for reproducibility
seed = 66
numpy.random.seed(seed)
#estimators = []
#estimators.append(('standardize', StandardScaler()))
#estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, nb_epoch=10, batch_size=10, verbose=1)))
# We define a pipeline of estimators, in which first the scaler is fitted to the data, then the MLP is applied
#pipeline = Pipeline(estimators)
#kfold = StratifiedKFold(y=Y_train, n_folds=3, shuffle=True, random_state=seed)

#model = create_baseline()
model = create_deeper_dropout_decay_PCA(k)
print model.summary()


____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
dropout_1 (Dropout)                (None, 100)         0           dropout_input_1[0][0]            
____________________________________________________________________________________________________
dense_1 (Dense)                    (None, 300)         30300       dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)                (None, 300)         0           dense_1[0][0]                    
____________________________________________________________________________________________________
dense_2 (Dense)                    (None, 300)         90300       dropout_2[0][0]                  
___________________________________________________________________________________________

In [11]:
# To save the best model
# serialize model to JSON
model_json = model.to_json()
with open(label+".model.json", "w") as json_file:
    json_file.write(model_json)
filepath=label+".weights.hdf5"
# Define that the accuracy in cv is monitored, and that weights are stored in a file when max accuracy is achieved
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Fit the model
history = model.fit(X_train_pca, dummy_y_train, validation_data=(X_test_pca,dummy_y_test), 
                    nb_epoch=500, batch_size=10, verbose=0, callbacks=callbacks_list)
#results = cross_val_score(pipeline, X_train, dummy_y_train, cv=kfold)
#print("Standardized data Acc (in CV training data): %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# evaluate the model
#scores = pipeline.evaluate(X_test, dummy_y_test)
#print pipeline.metrics_names[1]
#print scores[1]*100
# For other metrics, see http://machinelearningmastery.com/metrics-evaluate-machine-learning-algorithms-python/



Epoch 00000: val_acc improved from -inf to 0.34725, saving model to 7MLP-100PCA_et_GM_LOSO_Activity_1.weights.hdf5
Epoch 00001: val_acc did not improve
Epoch 00002: val_acc did not improve
Epoch 00003: val_acc did not improve
Epoch 00004: val_acc did not improve
Epoch 00005: val_acc improved from 0.34725 to 0.40659, saving model to 7MLP-100PCA_et_GM_LOSO_Activity_1.weights.hdf5
Epoch 00006: val_acc did not improve
Epoch 00007: val_acc improved from 0.40659 to 0.45714, saving model to 7MLP-100PCA_et_GM_LOSO_Activity_1.weights.hdf5
Epoch 00008: val_acc did not improve
Epoch 00009: val_acc did not improve
Epoch 00010: val_acc improved from 0.45714 to 0.46154, saving model to 7MLP-100PCA_et_GM_LOSO_Activity_1.weights.hdf5
Epoch 00011: val_acc did not improve
Epoch 00012: val_acc did not improve
Epoch 00013: val_acc did not improve
Epoch 00014: val_acc did not improve
Epoch 00015: val_acc improved from 0.46154 to 0.46813, saving model to 7MLP-100PCA_et_GM_LOSO_Activity_1.weights.hdf5
Epoch 

In [23]:
# Other performance/accuracy metrics
Y_pred = model.predict(X_test_pca)
print Y_pred.shape

# Accuracy
print('Accuracy:')
acc = accuracy_score(numpy.argmax(dummy_y_test, axis=1), numpy.argmax(Y_pred, axis=1))
print(acc)


# Confusion matrix
cm = confusion_matrix(numpy.argmax(dummy_y_test, axis=1), numpy.argmax(Y_pred, axis=1))
numpy.set_printoptions(precision=2)
print('Confusion matrix:')
print(cm)

# AUC
roc = roc_auc_score(dummy_y_test, Y_pred, average='macro')
print('AUC score:')
print(roc)

# F1
f1= f1_score(numpy.argmax(dummy_y_test, axis=1), numpy.argmax(Y_pred, axis=1), average='macro')
print('F1 score:')
print(f1)

# KAppa?
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(numpy.argmax(dummy_y_test, axis=1), numpy.argmax(Y_pred, axis=1))
print('Kappa score:')
print(kappa)

#import matplotlib
#matplotlib.use('Agg')
#import matplotlib.pyplot as plt

# summarize history for accuracy
#plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
#plt.title('model accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train','test'], loc='upper left')
#plt.show()
# summarize history for loss
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('model loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train','test'], loc='upper left')
#plt.show()


(455, 5)
Accuracy:
0.4
Confusion matrix:
[[19  0 11  6  0]
 [ 5 62 54 10 27]
 [13 28 53 27 20]
 [ 9  4 15 25  4]
 [ 0 13 25  2 23]]
AUC score:
0.70882891255
F1 score:
0.403064284675
Kappa score:
0.211498543162


In [24]:
# The models are stored already... what about the performance metrics?
# Store them in a csv
  
perfdata = [
    {'label': label, 'cm': json.dumps(cm.tolist()), 'acc': acc, 'auc': roc, 'f1': f1, 'kappa': kappa}
]
# Later, to recover cm:
# cm = np.array(json.loads(text))
df = pandas.DataFrame(perfdata)
filename = label+".perf.csv"
df.to_csv(filename, index=False, encoding='utf-8')